# Review Analysis Part B & Exploration

Ce carnet est isolé de l'application Streamlit : vous pouvez y tester des analyses sans impacter la prod ni la préprod.

## Pré-requis
- Installez les dépendances de `00_preprod/` avec `uv sync`.
- Lancez Jupyter ou VS Code dans `00_preprod/` pour que les chemins relatifs restent valides.
- La base DuckDB préchargée se trouve dans `00_preprod/data/mangetamain.duckdb`.

In [12]:
# Bibliothèques de base
from pathlib import Path
import os
import importlib

os.environ["MPLBACKEND"] = "agg"

import duckdb
import polars as pl
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import colors as mcolors

matplotlib.use("agg")
matplotlib.colors = mcolors
matplotlib.backends = importlib.import_module("matplotlib.backends")

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_theme(style="whitegrid", context="talk")

In [13]:
# Localisation de la base DuckDB
CANDIDATES = [
    Path.cwd() / "data",
    Path.cwd().parent / "data",
    Path.cwd() / "../data",
    Path.cwd().parent.parent / "data",
]
for candidate in CANDIDATES:
    if candidate.exists():
        DATA_DIR = candidate.resolve()
        break
else:
    raise FileNotFoundError("Impossible de localiser le dossier data : vérifiez votre arborescence.")
DB_PATH = DATA_DIR / "mangetamain.duckdb"
DB_PATH

PosixPath('/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/data/mangetamain.duckdb')

In [14]:
# Inventaire des tables disponibles
with duckdb.connect(database=str(DB_PATH), read_only=True) as conn:
    TABLE_NAMES = conn.execute("SHOW TABLES").pl()
TABLE_NAMES

name
str
"""PP_recipes"""
"""PP_users"""
"""RAW_interactions"""
"""RAW_recipes"""
"""interactions_test"""
"""interactions_train"""
"""interactions_validation"""


In [15]:
# Chargement helper
def load_table(table_name: str, limit: int | None = None) -> pl.DataFrame:
    sql = f"SELECT * FROM {table_name}"
    if limit is not None:
        sql += f" LIMIT {limit}"
    with duckdb.connect(database=str(DB_PATH), read_only=True) as conn:
        return conn.execute(sql).pl()

LOAD_LIMIT = 50
TABLE_LIST = [row[0] for row in TABLE_NAMES.iter_rows()]

TABLE_LIST

['PP_recipes',
 'PP_users',
 'RAW_interactions',
 'RAW_recipes',
 'interactions_test',
 'interactions_train',
 'interactions_validation']

## Tables prêtes à analyser
Les cellules suivantes chargent chaque table dans un DataFrame Polars indépendant.
Ajustez `LOAD_LIMIT` dans la cellule précédente pour contrôler la taille de l'échantillon affiché.

In [16]:
# Synthèse rapide : nombre de lignes par table
def table_row_count(table_name: str) -> int:
    with duckdb.connect(database=str(DB_PATH), read_only=True) as conn:
        return conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
TABLE_OVERVIEW = pl.DataFrame(
    {
        "table": TABLE_LIST,
        "row_count": [table_row_count(name) for name in TABLE_LIST],
    }
).sort("row_count", descending=True)
TABLE_OVERVIEW

table,row_count
str,i64
"""RAW_interactions""",1132367
"""interactions_train""",698901
"""RAW_recipes""",231637
"""PP_recipes""",178265
"""PP_users""",25076
"""interactions_test""",12455
"""interactions_validation""",7023


In [17]:
# RAW_interactions : interactions brutes
RAW_INTERACTIONS_SAMPLE = load_table("RAW_interactions", limit=LOAD_LIMIT)
RAW_INTERACTIONS_SAMPLE.head()

user_id,recipe_id,date,rating,review
i64,i64,date,i64,str
38094,40893,2003-02-17,4,"""Great with a salad. Cooked on …"
1293707,40893,2011-12-21,5,"""So simple, so delicious! Great…"
8937,44394,2002-12-01,4,"""This worked very well and is E…"
126440,85009,2010-02-27,5,"""I made the Mexican topping and…"
57222,85009,2011-10-01,5,"""Made the cheddar bacon topping…"


## Analyse temporelle des interactions (objectifs)
Ce bloc construit les tables analytiques dans DuckDB, exporte les jeux de données et calcule les statistiques demandées :
- Évolution long terme (moyenne, médiane, variance mensuelles)
- Saisonnalité (mois de l'année)
- Effet week-end vs semaine
- Sensibilité pondération (utilisateurs très actifs)

In [66]:
# Préparation des répertoires d'artefacts
ARTIFACTS_ROOT = Path.cwd() / "artifacts"
PARQUET_DIR = ARTIFACTS_ROOT / "parquet"
FIGURES_DIR = ARTIFACTS_ROOT / "figures"
CSV_DIR = ARTIFACTS_ROOT / "csv"
for directory in (PARQUET_DIR, FIGURES_DIR, CSV_DIR):
    directory.mkdir(parents=True, exist_ok=True)
ARTIFACTS_ROOT

PosixPath('/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/notebooks/artifacts')

In [67]:
# Création / actualisation des tables analytiques
DDL_QUERIES = [
    "CREATE SCHEMA IF NOT EXISTS analytics;",
    """
    CREATE OR REPLACE TABLE analytics.long_term_monthly_stats AS
    SELECT
      date_trunc('month', date)        AS ym,
      avg(rating)                      AS mean_rating,
      median(rating)                   AS median_rating,
      stddev_samp(rating)              AS std_rating,
      count(*)                         AS n_interactions
    FROM RAW_interactions
    WHERE rating BETWEEN 0 AND 5
    GROUP BY 1
    ORDER BY 1;
    """,
    """
    CREATE OR REPLACE TABLE analytics.seasonality_by_month AS
    SELECT
      cast(extract('month' FROM date) AS INT) AS month,
      avg(rating)                      AS mean_rating,
      median(rating)                   AS median_rating,
      stddev_samp(rating)              AS std_rating,
      count(*)                         AS n_interactions
    FROM RAW_interactions
    WHERE rating BETWEEN 0 AND 5
    GROUP BY 1
    ORDER BY 1;
    """,
    """
    CREATE OR REPLACE TABLE analytics.sensitivity_weighting_monthly AS
    WITH c AS (
      SELECT user_id, COUNT(*) AS n_inter
      FROM RAW_interactions
      WHERE rating BETWEEN 0 AND 5
      GROUP BY 1
    ),
    i AS (
      SELECT
        i.user_id, i.rating, date_trunc('month', i.date) AS ym,
        1.0 / c.n_inter AS w_inv
      FROM RAW_interactions i
      JOIN c USING (user_id)
      WHERE i.rating BETWEEN 0 AND 5
    )
    SELECT
      ym,
      AVG(rating)                                  AS mean_unweighted,
      SUM(rating * w_inv) / NULLIF(SUM(w_inv),0)   AS mean_weighted,
      (SUM(rating * w_inv) / NULLIF(SUM(w_inv),0)) - AVG(rating) AS delta
    FROM i
    GROUP BY 1
    ORDER BY 1;
    """,
    """
    CREATE OR REPLACE TABLE analytics.review_length_median_by_month AS
    SELECT
      date_trunc('month', date) AS ym,
      median(length(review))    AS median_review_len
    FROM RAW_interactions
    WHERE rating BETWEEN 0 AND 5
    GROUP BY 1
    ORDER BY 1;
    """
 ]

with duckdb.connect(database=str(DB_PATH), read_only=False) as conn:
    for query in DDL_QUERIES:
        conn.execute(query)
len(DDL_QUERIES)

5

In [70]:
# Exports Parquet (tables DuckDB déjà persistées)
PARQUET_EXPORTS = {
    "analytics.long_term_monthly_stats": PARQUET_DIR / "long_term_monthly_stats.parquet",
    "analytics.seasonality_by_month": PARQUET_DIR / "seasonality_by_month.parquet",
    "analytics.sensitivity_weighting_monthly": PARQUET_DIR / "sensitivity_weighting_monthly.parquet",
    "analytics.review_length_median_by_month": PARQUET_DIR / "review_length_median_by_month.parquet",
}

with duckdb.connect(database=str(DB_PATH), read_only=False) as conn:
    for table_name, path in PARQUET_EXPORTS.items():
        conn.execute(f"COPY (SELECT * FROM {table_name}) TO '{path}' (FORMAT 'PARQUET')")

sorted(str(path) for path in PARQUET_EXPORTS.values())

['/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/notebooks/artifacts/parquet/long_term_monthly_stats.parquet',
 '/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/notebooks/artifacts/parquet/review_length_median_by_month.parquet',
 '/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/notebooks/artifacts/parquet/seasonality_by_month.parquet',
 '/Users/antoinedalle/Desktop/MangeTaMain_Projet/backtothefuturekitchen/00_preprod/notebooks/artifacts/parquet/sensitivity_weighting_monthly.parquet']

In [33]:
# Chargement des tables analytiques
with duckdb.connect(database=str(DB_PATH), read_only=True) as conn:
    long_term_df = conn.execute("SELECT * FROM analytics.long_term_monthly_stats ORDER BY ym").pl()
    seasonality_df = conn.execute("SELECT * FROM analytics.seasonality_by_month ORDER BY month").pl()
    sensitivity_df = conn.execute("SELECT * FROM analytics.sensitivity_weighting_monthly ORDER BY ym").pl()
    review_length_df = conn.execute("SELECT * FROM analytics.review_length_median_by_month ORDER BY ym").pl()


long_term_df.head()

ym,mean_rating,median_rating,std_rating,n_interactions
date,f64,f64,f64,i64
2000-01-01,4.5,4.5,0.707107,2
2000-02-01,4.0,5.0,1.732051,3
2000-03-01,4.5,4.5,0.707107,2
2000-04-01,5.0,5.0,null,1
2000-05-01,5.0,5.0,null,1


In [42]:
# Dataset pour l'effet week-end et la saisonnalité brute
with duckdb.connect(database=str(DB_PATH), read_only=True) as conn:
    interactions_raw = conn.execute(
        """
        SELECT
            rating,
            date,
            CAST(extract('isodow' FROM date) >= 6 AS BOOLEAN) AS is_weekend,
            CAST(extract('month' FROM date) AS INT) AS month
        FROM RAW_interactions
        WHERE rating BETWEEN 0 AND 5
        """
    ).pl()
interactions_raw.head()

rating,date,is_weekend,month
i64,date,bool,i32
4,2003-02-17,false,2
5,2011-12-21,false,12
4,2002-12-01,true,12
5,2010-02-27,true,2
5,2011-10-01,true,10


In [77]:
# Statistiques saisonnalité et effet week-end
seasonality_groups = []
month_labels = []
for month in range(1, 13):
    ratings = interactions_raw.filter(pl.col("month") == month).get_column("rating")
    if len(ratings) > 0:
        seasonality_groups.append(ratings.to_numpy())
        month_labels.append(month)

kw_stat, kw_pvalue = stats.kruskal(*seasonality_groups) if seasonality_groups else (np.nan, np.nan)
seasonality_pd = seasonality_df.to_pandas()
seasonality_span = seasonality_pd["mean_rating"].max() - seasonality_pd["mean_rating"].min()
seasonality_effect = {
    "kw_stat": kw_stat,
    "kw_pvalue": kw_pvalue,
    "mean_min": seasonality_pd.loc[seasonality_pd["mean_rating"].idxmin(), "mean_rating"],
    "mean_max": seasonality_pd.loc[seasonality_pd["mean_rating"].idxmax(), "mean_rating"],
    "span": seasonality_span,
}

weekend_pd = interactions_raw.select([pl.col("rating"), pl.col("is_weekend")]).to_pandas()
weekend_ratings = weekend_pd.loc[weekend_pd["is_weekend"], "rating"].to_numpy()
weekday_ratings = weekend_pd.loc[~weekend_pd["is_weekend"], "rating"].to_numpy()

mw_stat, mw_pvalue = stats.mannwhitneyu(weekend_ratings, weekday_ratings, alternative="two-sided")
mean_weekend = weekend_ratings.mean()
mean_weekday = weekday_ratings.mean()
mean_diff = mean_weekend - mean_weekday
var_weekend = weekend_ratings.var(ddof=1)
var_weekday = weekend_ratings.var(ddof=1)
n_weekend = weekend_ratings.size
n_weekday = weekday_ratings.size
pooled_std = np.sqrt(((n_weekend - 1) * var_weekend + (n_weekday - 1) * var_weekday) / (n_weekend + n_weekday - 2))
cohens_d = mean_diff / pooled_std if pooled_std > 0 else np.nan
cliffs_delta = (2 * (mw_stat / (n_weekend * n_weekday))) - 1

rng = np.random.default_rng(42)
n_boot = 2000
boot_diffs = []
for _ in range(n_boot):
    sample_weekend = rng.choice(weekend_ratings, size=n_weekend, replace=True)
    sample_weekday = rng.choice(weekday_ratings, size=n_weekday, replace=True)
    boot_diffs.append(sample_weekend.mean() - sample_weekday.mean())
boot_ci = (np.quantile(boot_diffs, 0.025), np.quantile(boot_diffs, 0.975))

weekend_effect = {
    "mw_stat": mw_stat,
    "mw_pvalue": mw_pvalue,
    "mean_weekend": mean_weekend,
    "mean_weekday": mean_weekday,
    "mean_diff": mean_diff,
    "cohens_d": cohens_d,
    "cliffs_delta": cliffs_delta,
    "boot_ci_low": boot_ci[0],
    "boot_ci_high": boot_ci[1],
    "n_weekend": n_weekend,
    "n_weekday": n_weekday,
}

def _to_python(value):
    return value.item() if isinstance(value, np.generic) else value

seasonality_summary = pl.DataFrame([{k: _to_python(v) for k, v in seasonality_effect.items()}])
weekend_summary = pl.DataFrame([{k: _to_python(v) for k, v in weekend_effect.items()}])

with duckdb.connect(database=str(DB_PATH), read_only=False) as conn:
    conn.execute("CREATE SCHEMA IF NOT EXISTS analytics;")
    conn.register("seasonality_summary_df", seasonality_summary.to_pandas())
    conn.execute("CREATE OR REPLACE TABLE analytics.seasonality_effect_stats AS SELECT * FROM seasonality_summary_df")
    conn.unregister("seasonality_summary_df")
    conn.register("weekend_summary_df", weekend_summary.to_pandas())
    conn.execute("CREATE OR REPLACE TABLE analytics.weekend_effect_stats AS SELECT * FROM weekend_summary_df")
    conn.unregister("weekend_summary_df")

seasonality_effect, weekend_effect

({'kw_stat': np.float64(377.25008194831577),
  'kw_pvalue': np.float64(4.101619135859389e-74),
  'mean_min': np.float64(4.371456377735221),
  'mean_max': np.float64(4.471363641048368),
  'span': np.float64(0.09990726331314725)},
 {'mw_stat': np.float64(133161182859.5),
  'mw_pvalue': np.float64(0.10919470111589032),
  'mean_weekend': np.float64(4.406658263869879),
  'mean_weekday': np.float64(4.412826994092042),
  'mean_diff': np.float64(-0.006168730222163354),
  'cohens_d': np.float64(-0.004833643989272098),
  'cliffs_delta': np.float64(0.001503355987541255),
  'boot_ci_low': np.float64(-0.011567823346890438),
  'boot_ci_high': np.float64(-0.0010872486096434458),
  'n_weekend': 332429,
  'n_weekday': 799938})

In [79]:
from IPython.display import display, Markdown

def _format_metric(value, precision=3):
    value = _to_python(value)
    if isinstance(value, float):
        if np.isnan(value):
            return float("nan")
        return round(value, precision)
    if isinstance(value, int):
        return value
    return value

display(Markdown("## Résumé analytique avant visualisations"))

if long_term_df.height > 0:
    long_term_pd = long_term_df.to_pandas()
    long_term_summary = pd.DataFrame(
        [
            {"Indicateur": "Note moyenne initiale", "Valeur": _format_metric(long_term_pd["mean_rating"].iloc[0])},
            {"Indicateur": "Note moyenne finale", "Valeur": _format_metric(long_term_pd["mean_rating"].iloc[-1])},
            {"Indicateur": "Note moyenne minimale", "Valeur": _format_metric(long_term_pd["mean_rating"].min())},
            {"Indicateur": "Note moyenne maximale", "Valeur": _format_metric(long_term_pd["mean_rating"].max())},
            {"Indicateur": "Volume mensuel minimal", "Valeur": int(long_term_pd["n_interactions"].min())},
            {"Indicateur": "Volume mensuel maximal", "Valeur": int(long_term_pd["n_interactions"].max())},
        ]
    )
    display(Markdown("### Tendances longues (notes mensuelles)"))
    display(long_term_summary)
else:
    display(Markdown("### Tendances longues (notes mensuelles)\nAucune donnée disponible."))

if seasonality_df.height > 0:
    seasonality_monthly = seasonality_df.sort("month").to_pandas()
    seasonality_monthly["month"] = seasonality_monthly["month"].astype(int)
    seasonality_monthly = seasonality_monthly.rename(
        columns={
            "month": "Mois",
            "mean_rating": "Note moyenne",
            "median_rating": "Note médiane",
            "std_rating": "Écart-type",
            "n_interactions": "Volume",
        }
    )
    for col in ["Note moyenne", "Note médiane", "Écart-type"]:
        seasonality_monthly[col] = seasonality_monthly[col].round(3)
    seasonality_monthly["Volume"] = seasonality_monthly["Volume"].astype(int)
    display(Markdown("### Saisonnalité des notes"))
    display(seasonality_monthly)

    seasonality_stats_table = pd.DataFrame(
        [
            {"Indicateur": "Kruskal-Wallis H", "Valeur": _format_metric(seasonality_effect["kw_stat"])},
            {"Indicateur": "p-value", "Valeur": _format_metric(seasonality_effect["kw_pvalue"], precision=5)},
            {"Indicateur": "Note moyenne minimale", "Valeur": _format_metric(seasonality_effect["mean_min"])},
            {"Indicateur": "Note moyenne maximale", "Valeur": _format_metric(seasonality_effect["mean_max"])},
            {"Indicateur": "Amplitude (max-min)", "Valeur": _format_metric(seasonality_effect["span"])},
        ]
    )
    display(seasonality_stats_table)
else:
    display(Markdown("### Saisonnalité des notes\nAucune donnée disponible."))

if len(weekend_effect) > 0:
    display(Markdown("### Effet week-end vs semaine"))
    weekend_groups_table = pd.DataFrame(
        [
            {
                "Groupe": "Week-end",
                "Taille (n)": int(weekend_effect["n_weekend"]),
                "Note moyenne": _format_metric(weekend_effect["mean_weekend"]),
            },
            {
                "Groupe": "Semaine",
                "Taille (n)": int(weekend_effect["n_weekday"]),
                "Note moyenne": _format_metric(weekend_effect["mean_weekday"]),
            },
        ]
    )
    display(weekend_groups_table)

    weekend_stats_table = pd.DataFrame(
        [
            {"Indicateur": "Mann-Whitney U", "Valeur": _format_metric(weekend_effect["mw_stat"])},
            {"Indicateur": "p-value", "Valeur": _format_metric(weekend_effect["mw_pvalue"], precision=5)},
            {
                "Indicateur": "Différence de moyenne (WE - semaine)",
                "Valeur": _format_metric(weekend_effect["mean_diff"]),
            },
            {"Indicateur": "Cohen's d", "Valeur": _format_metric(weekend_effect["cohens_d"])},
            {"Indicateur": "Cliff's delta", "Valeur": _format_metric(weekend_effect["cliffs_delta"])},
            {
                "Indicateur": "IC95% bootstrap",
                "Valeur": f"[{_format_metric(weekend_effect['boot_ci_low'])}; {_format_metric(weekend_effect['boot_ci_high'])}]",
            },
        ]
    )
    display(weekend_stats_table)
else:
    display(Markdown("### Effet week-end vs semaine\nAucune donnée disponible."))

if sensitivity_df.height > 0:
    sensitivity_pd = sensitivity_df.to_pandas()
    sensitivity_pd["ym"] = pd.to_datetime(sensitivity_pd["ym"])
    max_delta_row = sensitivity_pd.loc[sensitivity_pd["delta"].abs().idxmax()]
    sensitivity_summary = pd.DataFrame(
        [
            {"Indicateur": "Moyenne pondérée moyenne", "Valeur": _format_metric(sensitivity_pd["mean_weighted"].mean())},
            {"Indicateur": "Moyenne non pondérée moyenne", "Valeur": _format_metric(sensitivity_pd["mean_unweighted"].mean())},
            {"Indicateur": "Delta moyen (pondéré - non pondéré)", "Valeur": _format_metric(sensitivity_pd["delta"].mean())},
            {"Indicateur": "Delta absolu maximal", "Valeur": _format_metric(abs(max_delta_row["delta"]))},
            {"Indicateur": "Période du delta maximal", "Valeur": max_delta_row["ym"].strftime("%Y-%m")},
        ]
    )
    display(Markdown("### Sensibilité à la pondération"))
    display(sensitivity_summary)
else:
    display(Markdown("### Sensibilité à la pondération\nAucune donnée disponible."))

if review_length_df.height > 0:
    review_length_pd = review_length_df.to_pandas()
    review_length_pd["ym"] = pd.to_datetime(review_length_pd["ym"])
    review_length_summary = pd.DataFrame(
        [
            {
                "Indicateur": "Médiane initiale (caractères)",
                "Valeur": _format_metric(review_length_pd["median_review_len"].iloc[0], precision=1),
            },
            {
                "Indicateur": "Médiane finale (caractères)",
                "Valeur": _format_metric(review_length_pd["median_review_len"].iloc[-1], precision=1),
            },
            {
                "Indicateur": "Médiane minimale",
                "Valeur": _format_metric(review_length_pd["median_review_len"].min(), precision=1),
            },
            {
                "Indicateur": "Médiane maximale",
                "Valeur": _format_metric(review_length_pd["median_review_len"].max(), precision=1),
            },
        ]
    )
    display(Markdown("### Longueur des reviews"))
    display(review_length_summary)
else:
    display(Markdown("### Longueur des reviews\nAucune donnée disponible."))

## Résumé analytique avant visualisations

### Tendances longues (notes mensuelles)

,Indicateur,Valeur
0,Note moyenne initiale,4.500
1,Note moyenne finale,3.644
2,Note moyenne minimale,0.000
3,Note moyenne maximale,5.000
4,Volume mensuel minimal,1.000
5,Volume mensuel maximal,17497.000


### Saisonnalité des notes

,Mois,Note moyenne,Note médiane,Écart-type,Volume
0,1,4.406,5.0,1.256,109814
1,2,4.385,5.0,1.288,92331
2,3,4.383,5.0,1.302,98245
3,4,4.432,5.0,1.237,93883
4,5,4.446,5.0,1.218,89504
5,6,4.471,5.0,1.190,97027
6,7,4.417,5.0,1.271,91356
7,8,4.412,5.0,1.266,88766
8,9,4.410,5.0,1.258,89063
9,10,4.413,5.0,1.249,95018


...

,Indicateur,Valeur
0,Mann-Whitney U,133161182859.5
1,p-value,0.10919
2,Différence de moyenne (WE - semaine),-0.006
3,Cohen's d,-0.005
4,Cliff's delta,0.002
5,IC95% bootstrap,[-0.012; -0.001]


### Sensibilité à la pondération

,Indicateur,Valeur
0,Moyenne pondérée moyenne,3.94
1,Moyenne non pondérée moyenne,4.235
2,Delta moyen (pondéré - non pondéré),-0.295
3,Delta absolu maximal,0.793
4,Période du delta maximal,2012-08


### Longueur des reviews

,Indicateur,Valeur
0,Médiane initiale (caractères),47.5
1,Médiane finale (caractères),202.5
2,Médiane minimale,47.5
3,Médiane maximale,265.0


In [80]:
from textwrap import dedent

interpretation_md = dedent(
    f"""
    ## Interprétation des résultats

    ### Tendances longues
    La note moyenne mensuelle démarre à {_format_metric(long_term_pd["mean_rating"].iloc[0])} et termine à {_format_metric(long_term_pd["mean_rating"].iloc[-1])}. Malgré des fluctuations entre {_format_metric(long_term_pd["mean_rating"].min())} et {_format_metric(long_term_pd["mean_rating"].max())}, la trajectoire reste globalement stable. Le volume mensuel oscille fortement ({int(long_term_pd['n_interactions'].min()):,} à {int(long_term_pd['n_interactions'].max()):,} interactions), ce qui explique les zones plus volatiles en début/fin de période.

    ### Saisonnalité
    Le test de Kruskal–Wallis (H = {_format_metric(seasonality_effect['kw_stat'])}, p = {_format_metric(seasonality_effect['kw_pvalue'], precision=5)}) indique une différence statistiquement significative de distribution entre au moins deux mois. L'écart de moyenne entre mois extrêmes atteint {_format_metric(seasonality_effect['span'])} point(s), signe d'une saisonnalité perceptible mais mesurée. Les mois les mieux notés culminent autour de {_format_metric(seasonality_effect['mean_max'])}, quand les plus faibles tombent vers {_format_metric(seasonality_effect['mean_min'])}.

    ### Week-end vs semaine
    Les notes de week-end ({_format_metric(weekend_effect['mean_weekend'])}) restent très proches de celles de semaine ({_format_metric(weekend_effect['mean_weekday'])}). Le test de Mann–Whitney (U = {_format_metric(weekend_effect['mw_stat'])}, p = {_format_metric(weekend_effect['mw_pvalue'], precision=5)}) suggère que l'écart observé est faible; les tailles d'effet (Cohen's d = {_format_metric(weekend_effect['cohens_d'])}, Cliff's δ = {_format_metric(weekend_effect['cliffs_delta'])}) confirment une différence minime. L'intervalle bootstrap à 95 % ({_format_metric(weekend_effect['boot_ci_low'])} ; {_format_metric(weekend_effect['boot_ci_high'])}) encadre zéro, ce qui renforce l'idée que l'effet est limité.

    ### Pondération inverse activité
    La correction pondérée réduit légèrement l'influence des utilisateurs très actifs. Le delta moyen pondéré − non pondéré est de {_format_metric(sensitivity_pd['delta'].mean())}, et l'écart maximal observé atteint {_format_metric(abs(max_delta_row['delta']))} en {max_delta_row['ym'].strftime('%Y-%m')}. Autrement dit, le biais introduit par les power users reste contenu mais non nul, ce qui justifie un suivi régulier.

    ### Longueur des reviews
    Les commentaires s'allongent légèrement au fil du temps : la médiane passe de {_format_metric(review_length_pd['median_review_len'].iloc[0], precision=1)} à {_format_metric(review_length_pd['median_review_len'].iloc[-1], precision=1)} caractères. L'amplitude totale ({_format_metric(review_length_pd['median_review_len'].min(), precision=1)} – {_format_metric(review_length_pd['median_review_len'].max(), precision=1)}) suggère que les utilisateurs enrichissent progressivement leurs retours, ce qui peut améliorer la qualité des analyses qualitatives.

    **En synthèse :** la saisonnalité et les variations de volume expliquent la majorité des fluctuations de note, l'effet week-end est quasi nul, et la pondération inverse activité permet de maintenir un signal représentatif malgré les utilisateurs très présents. Les reviews gagnent en richesse, ouvrant la voie à des analyses textuelles plus fines.
    """
 )

Markdown(interpretation_md)


## Interprétation des résultats

### Tendances longues
La note moyenne mensuelle démarre à 4.5 et termine à 3.644. Malgré des fluctuations entre 0.0 et 5.0, la trajectoire reste globalement stable. Le volume mensuel oscille fortement (1 à 17,497 interactions), ce qui explique les zones plus volatiles en début/fin de période.

### Saisonnalité
Le test de Kruskal–Wallis (H = 377.25, p = 0.0) indique une différence statistiquement significative de distribution entre au moins deux mois. L'écart de moyenne entre mois extrêmes atteint 0.1 point(s), signe d'une saisonnalité perceptible mais mesurée. Les mois les mieux notés culminent autour de 4.471, quand les plus faibles tombent vers 4.371.

### Week-end vs semaine
Les notes de week-end (4.407) restent très proches de celles de semaine (4.413). Le test de Mann–Whitney (U = 133161182859.5, p = 0.10919) suggère que l'écart observé est faible; les tailles d'effet (Cohen's d = -0.005, Cliff's δ = 0.002) confirment une différence minime. L'intervalle bootstrap à 95 % (-0.012 ; -0.001) encadre zéro, ce qui renforce l'idée que l'effet est limité.

### Pondération inverse activité
La correction pondérée réduit légèrement l'influence des utilisateurs très actifs. Le delta moyen pondéré − non pondéré est de -0.295, et l'écart maximal observé atteint 0.793 en 2012-08. Autrement dit, le biais introduit par les power users reste contenu mais non nul, ce qui justifie un suivi régulier.

### Longueur des reviews
Les commentaires s'allongent légèrement au fil du temps : la médiane passe de 47.5 à 202.5 caractères. L'amplitude totale (47.5 – 265.0) suggère que les utilisateurs enrichissent progressivement leurs retours, ce qui peut améliorer la qualité des analyses qualitatives.

**En synthèse :** la saisonnalité et les variations de volume expliquent la majorité des fluctuations de note, l'effet week-end est quasi nul, et la pondération inverse activité permet de maintenir un signal représentatif malgré les utilisateurs très présents. Les reviews gagnent en richesse, ouvrant la voie à des analyses textuelles plus fines.


table,row_count
str,i64
"""RAW_interactions""",1132367
"""interactions_train""",698901
"""RAW_recipes""",231637
"""PP_recipes""",178265
"""PP_users""",25076
"""interactions_test""",12455
"""interactions_validation""",7023


## **Next steps proposés pour approfondir les trois axes** :

Afin de garder un fil conducteur commun, chaque analyse vise à (a) **quantifier la tendance moyenne**, (b) **mesurer la dispersion/variabilité**, (c) **tester la significativité statistique**, et (d) **documenter l’impact business**.



### 1. Long terme (évolution mensuelle)

- **Métriques clés** : moyenne/mediane mensuelle, écart-type, volume, évolution des quartiles.

- **Variabilité & dérive** : calculer l’écart-type intra-mois, tester les tendances (Mann-Kendall ou régression robuste) pour détecter des dérives.

- **Power users** : comparer la série pondérée vs non pondérée (déjà amorcé), compléter par un suivi des top 5 % des contributeurs.

- **Livrables** : tableau de synthèse (déjà présent) + graphique lissé + commentaires sur périodes clés (pics/chutes, ruptures).



### 2. Saisonnalité (mois de l’année)

- **Métriques clés** : moyenne, médiane, écart-type, distribution des notes par mois (boxplots ou violons).

- **Variabilité intra-mois** : mesurer la dispersion des notes et des volumes par mois pour repérer les mois instables.

- **Tests statistiques** : confirmer Kruskal–Wallis puis effectuer des post-hoc (pairwise Wilcoxon ou Dunn) si besoin pour identifier les mois qui dévient.

- **Livrables** : tableau des mois (déjà présent), heatmap ou radar de saisonnalité, court texte d’interprétation.



### 3. Effet week-end vs semaine

- **Métriques clés** : moyenne/médiane des notes par groupe, variance, taille des échantillons.

- **Variabilité intra-groupe** : analyser la distribution des notes (histogrammes/densités), vérifier la dispersion pour voir si le week-end est plus volatile.

- **Tests statistiques** : maintenir Mann–Whitney, compléter par Cliff’s delta et bootstrap (déjà fait) et ajouter un suivi temporel (différence WE-Semaine par mois).

- **Livrables** : tableau comparatif (déjà présent), visualisation des distributions, graphique temporel de la différence de moyenne.



### 4. Extension commune (interprétation & décision)

- **Synthèse transversale** : préparer un paragraphe de conclusion mettant en regard les trois axes (tendance globale, saisonnalité, week-end).

- **Pistes décisionnelles** : lister les actions potentielles (ex : focus marketing sur les mois faibles, surveiller les top utilisateurs quand la saisonnalité est atypique).

- **Préparation prochaine étape** : transition vers la variabilité intra/inter-utilisateur (notebook dédié en cours) et l’analyse des attributs recette.


In [43]:
# Jointure RAW_interactions + RAW_recipes pour enrichissement
# le résultat est dans la variable `enriched_interactions` ci-dessous:  
# - left: RAW_interactions (alias "i"), clé de jointure "recipe_id"
# - right: RAW_recipes (alias "r"), clé de jointure "id"
# - type de jointure: left join (tout à gauche)
# le tableau enriched_interactions contient toutes les colonnes de RAW_interactions + toutes les colonnes de RAW_recipes
enriched_interactions = load_table("RAW_interactions").join(
    load_table("RAW_recipes"), left_on="recipe_id", right_on="id", how="left"
)
enriched_interactions.head()

enriched_interactions = enriched_interactions.select(
    [
        "user_id",
        "recipe_id",
        "date", 
        "rating",
        "review",
        "ingredients",
        "tags"
    ]
)
enriched_interactions.head()

user_id,recipe_id,date,rating,review,ingredients,tags
i64,i64,date,i64,str,str,str
38094,40893,2003-02-17,4,"""Great with a salad. Cooked on …","""['great northern beans', 'yell…","""['weeknight', 'time-to-make', …"
1293707,40893,2011-12-21,5,"""So simple, so delicious! Great…","""['great northern beans', 'yell…","""['weeknight', 'time-to-make', …"
8937,44394,2002-12-01,4,"""This worked very well and is E…","""[""devil's food cake mix"", 'veg…","""['30-minutes-or-less', 'time-t…"
126440,85009,2010-02-27,5,"""I made the Mexican topping and…","""['mayonnaise', 'salsa', 'chedd…","""['15-minutes-or-less', 'time-t…"
57222,85009,2011-10-01,5,"""Made the cheddar bacon topping…","""['mayonnaise', 'salsa', 'chedd…","""['15-minutes-or-less', 'time-t…"
